In [74]:
import pandas as pd
import numpy as np
import time

In [75]:
#read in the list of videos contained in the csv
df = pd.read_csv('vidz.csv')

In [76]:
#drop columns with redundant information
df = df.drop(columns=['comments_disabled', 'ratings_disabled' ,'thumbnail_link'])

In [78]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   video_id       200 non-null    object 
 1   title          200 non-null    object 
 2   publishedAt    200 non-null    object 
 3   channelTitle   200 non-null    object 
 4   categoryId     200 non-null    int64  
 5   trending_date  200 non-null    object 
 6   tags           200 non-null    object 
 7   view_count     200 non-null    int64  
 8   likes          200 non-null    int64  
 9   comment_count  199 non-null    float64
 10  description    200 non-null    object 
 11  duration       200 non-null    object 
dtypes: float64(1), int64(3), object(8)
memory usage: 18.9+ KB


## Convert to datetime --> ppublised at, trending date, duration (timedelta)

In [79]:
#convert publishedAT to datetime column
df.publishedAt = pd.to_datetime(df.publishedAt)
df.loc[1,'publishedAt']

Timestamp('2022-10-26 15:58:27+0000', tz='UTC')

In [80]:
#convert trending to datetime column
df.trending_date = pd.to_datetime(df.trending_date, format='%y.%d.%m')
df.trending_date

0     2022-10-27
1     2022-10-27
2     2022-10-27
3     2022-10-27
4     2022-10-27
         ...    
195   2022-10-27
196   2022-10-27
197   2022-10-27
198   2022-10-27
199   2022-10-27
Name: trending_date, Length: 200, dtype: datetime64[ns]

In [81]:
#import tzinfo inorder to strip time zone information from published at. 
#this makes it a "naive" datetime object. may want to change this approach
from datetime import tzinfo

df.loc[1,'publishedAt'].replace(tzinfo=None)

Timestamp('2022-10-26 15:58:27')

In [82]:
#strips the timezone from each row
for n in range(0,200):
    df.loc[n,'publishedAt']= df.loc[n,'publishedAt'].replace(tzinfo=None)

In [83]:
df.loc[10, ['publishedAt']]

publishedAt    2022-10-27 04:04:06
Name: 10, dtype: object

In [105]:
#create age column. may want to give timezone info to trending date instead of removing it from pblishedAt
df['age']=(df.trending_date - df.publishedAt)

/opt/homebrew/anaconda3/lib/python3.9/site-packages/pandas/core/arrays/datetimelike.py:1189: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  warnings.warn(


# View:Like ratio that can score the video | view:comment ratio

Have them all as a weighted ratio

views = views - likes - comments

engagement = views + (likes * 2) + (comments * 4) --> n==measure of effort. view and click = like ... view, click, type, submit = comment


In [85]:
#creates engagement metric. not sure how to do weightd columns really
df['engagement'] = (df.view_count - df.likes) + (df.likes * 2) + (df.comment_count * 4) 

In [86]:
df.engagement

0        851453.0
1       1812843.0
2       1057839.0
3        514531.0
4       1624979.0
          ...    
195      909446.0
196     6407837.0
197    13915892.0
198     1064503.0
199     1990537.0
Name: engagement, Length: 200, dtype: float64

## Is the video sponsored?


In [87]:
#adds sponsored column based on appearance of word sponsored in the description
df['sponsored'] = np.where(df.description.str.contains('sponsor'), 'yes', 'no')
df[df.description.str.contains('sponsored')].shape

## Video language


Percent of capital letters in title 


Age restricted


When we talk about subscribers we can take a look at age of channel


How to drive up subscribers is a slightly separate question but we can ask it


At what point does the video view count pass the subscriber view count


We can target placing videos in/out of the top 25 


## Let’s think about tags and how many words they have in common with the descriptions


In [97]:
#countes number of tags given to video BEFORE stripping out extraneous things
df['num_of_tags'] = df.tags.str.split('|').str.len()

In [98]:
#gets rid of separator
df.tags = df.tags.str.replace('|'," ")

/var/folders/x7/9w9mh0fd73zg4jr8l_9v9h840000gn/T/ipykernel_2496/2021686076.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.tags = df.tags.str.replace('|'," ")


In [99]:
#Imports spacy
import spacy

In [100]:
#creates the nlp object that is going to do the heavy lifting
nlp = spacy.load("en_core_web_sm")

In [101]:
#uses the nlp object to convert the input text into a doc
doc = nlp(df.loc[0].tags)

In [102]:
#goes through tokens (words) in each doc
for token in doc:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

# Iterate over the predicted entities
for ent in doc.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

lil         PROPN     nmod      
durk        PROPN     amod      
lil         PROPN     nmod      
durk        PROPN     compound  
music       PROPN     nmod      
lil         PROPN     nummod    
durk        PROPN     compound  
music       NOUN      compound  
video       NOUN      nsubj     
just        ADV       advmod    
cause       VERB      mark      
y           PRON      nsubj     
all         PRON      appos     
waited      VERB      ROOT      
2           NUM       nummod    
lil         NOUN      compound  
durk        PROPN     dobj      
2020        NUM       nummod    
just        ADV       advmod    
cause       VERB      mark      
y'          PRON      nsubj     
all         PRON      appos     
waited      VERB      advcl     
2           NUM       nummod    
durkio      NOUN      dobj      
smurkio     VERB      conj      
lil         PROPN     compound  
durk        PROPN     amod      
official    ADJ       amod      
drill       NOUN      dobj      
drill     

In [107]:
#creates set of unique words in doc 
tokens = set()
for token in doc:
    tokens.add(token.text)
print(tokens)

{'chicago', "y'", 'OTF', '2020', 'video', 'hip', 'official', 'hop', 'drill', 'the', 'waited', 'smurkio', 'lil', 'only', 'music', '2', 'durkio', 'family', 'y', 'just', 'all', 'durk', 'cause'}


In [135]:
def clean_text(text):
    """ 
    Purpose:
        to clean text input into function by removing duplicate words, punctuations, and other things
    ---
    Parameters:
        text: a string
    ---
    Returns:
        tokens: a set of words found in the input text
    """

    nlp = spacy.load("en_core_web_sm")
    doc = nlp(text)
    tokens = set()
    
    for token in doc:
        if token.pos_ not in ['SYM', 'PUNCT', 'DET']:
            tokens.add(token.text)

    for ent in doc.ents:
        tokens.add(ent.text)

    return tokens
    
    

In [116]:
docb = nlp(df.loc[4].description)

In [117]:
for token in docb:
    # Get the token text, part-of-speech tag and dependency label
    token_text = token.text
    token_pos = token.pos_
    token_dep = token.dep_
    # This is for formatting only
    print(f"{token_text:<12}{token_pos:<10}{token_dep:<10}")

# Iterate over the predicted entities
for ent in docb.ents:
    # Print the entity text and its label
    print(ent.text, ent.label_)

The         DET       det       
Inside      PROPN     compound  
fellas      NOUN      nsubj     
break       VERB      ROOT      
down        ADP       prt       
the         DET       det       
Klay        PROPN     compound  
-           PUNCT     punct     
Dbook       NOUN      compound  
interaction NOUN      dobj      
and         CCONJ     cc        
recap       VERB      conj      
Warriors    PROPN     compound  
-           PUNCT     punct     
Suns        PROPN     dobj      
on          ADP       prep      
TNT         PROPN     pobj      
.           PUNCT     punct     
Watch       VERB      ROOT      
highlights  NOUN      dobj      
from        ADP       prep      
Inside      ADP       prep      
the         DET       det       
NBA         PROPN     pobj      
with        ADP       prep      
Shaq        PROPN     pobj      
,           PUNCT     punct     
Charles     PROPN     compound  
Barkley     PROPN     conj      
,           PUNCT     punct     
Kenny     

In [134]:
tokens = set()
for token in docb:
    if token.pos_ not in ['SYM', 'PUNCT', 'DET']:
        print(token.pos_)
        tokens.add(token.text)

for ent in docb.ents:
    print(ent.text, ent.label_)
print(tokens)

PROPN
NOUN
VERB
ADP
PROPN
NOUN
NOUN
CCONJ
VERB
PROPN
PROPN
ADP
PROPN
VERB
NOUN
ADP
ADP
PROPN
ADP
PROPN
PROPN
PROPN
PROPN
PROPN
CCONJ
PROPN
PROPN
CCONJ
ADJ
VERB
ADV
PART
AUX
VERB
ADP
ADJ
NOUN
X
SPACE
PROPN
ADP
PROPN
ADP
PROPN
SPACE
VERB
PROPN
ADP
PROPN
ADP
PROPN
NOUN
SPACE
ADP
PROPN
ADP
PROPN
ADP
PROPN
NOUN
SPACE
VERB
PROPN
ADP
PROPN
ADP
PROPN
X
Warriors-Suns ORG
TNT ORG
NBA ORG
Charles Barkley PERSON
Kenny Smith PERSON
Ernie Johnson PERSON
NBA ORG
TNT ORG
TNT ORG
NBA ORG
TNT ORG
TNT ORG
Instagram ORG
{'Smith', 'updated', 'Suns', 'Subscribe', 'Twitter', 'Klay', 'on', 'highlights', 'NBA', 'Charles', 'break', 'Facebook', 'to', 'fellas', 'Johnson', 'Connect', 'Warriors', 'recap', 'down', 'TNT', 'Kenny', 'Barkley', 'Shaq', 'Ernie', 'https://twitter.com/NBAonTNT', 'Instagram', 'https://www.youtube.com/nbaontnt?sub_confirmation=1', 'Follow', 'https://www.facebook.com/NBAONTNT/', 'Like', 'Dbook', 'from', 'with', 'more', 'https://www.instagram.com/nbaontnt/?hl=en', 'Watch', 'Inside', 'interacti

In [133]:
for ent in docb.ents:
    print(ent.text, ent.label_)

Warriors-Suns ORG
TNT ORG
NBA ORG
Charles Barkley PERSON
Kenny Smith PERSON
Ernie Johnson PERSON
NBA ORG
TNT ORG
TNT ORG
NBA ORG
TNT ORG
TNT ORG
Instagram ORG


In [137]:
df['cleaned_desc'] = df['description'].apply(lambda x: clean_text(x))

## Think about combining the countries top 25 lists and control for duplicates. This way we can classify what videos have been a top 25 video 
